In [414]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

#import realtime trades csv
dfa = pd.read_csv(r'insert your first file path here',
                   usecols = [ 'Strategy', 'Symbol', 'DateIn','PriceIn',  'PriceOut','DateOut', 'Reason'], parse_dates=False)

#set dateout as date to filter out dates
dfa.DateOut = pd.to_datetime(dfa.DateOut)

dfa

,Strategy,Symbol,DateIn,PriceIn,DateOut,PriceOut,Reason
0,simple_day_trade,EDU,3/17/21,16.70,2021-03-17,16.72,exit rule
1,simple_day_trade,CLDR-202110,3/17/21,12.42,2021-03-17,12.79,exit rule
2,simple_day_trade,FCCY,3/17/21,18.36,2021-03-17,18.72,exit rule
3,simple_day_trade,FOSL,3/17/21,13.38,2021-03-17,14.09,exit rule
4,simple_day_trade,GOGO,3/17/21,10.36,2021-03-17,10.79,exit rule
...,...,...,...,...,...,...,...
2343,simple_day_trade,CUE,11/29/21,12.16,2021-11-29,12.01,exit rule
2344,simple_day_trade,IONQ,11/29/21,22.76,2021-11-29,24.08,exit rule
2345,simple_day_trade,OSTK,11/29/21,89.78,2021-11-29,90.06,exit rule
2346,simple_day_trade,RDIB,11/29/21,27.82,2021-11-29,27.16,exit rule


In [408]:
#import model trades csv
dfm = pd.read_csv(r'insert your second file path here',
                   usecols = ['Strategy', 'Symbol', 'DateIn','PriceIn', 'PriceOut', 'DateOut', 'Reason'], parse_dates=False)

#set dateout as date to filter out dates
dfm.DateOut=pd.to_datetime(dfm.DateOut)

dfm

,Strategy,Symbol,DateIn,PriceIn,DateOut,PriceOut,Reason
0,simple_day_trade,APA,3/17/21,20.44,2021-03-17,20.99,exit rule
1,simple_day_trade,CLR,3/17/21,26.82,2021-03-17,27.45,exit rule
2,simple_day_trade,ESRT,3/17/21,11.36,2021-03-17,11.46,exit rule
3,simple_day_trade,LW,3/17/21,79.44,2021-03-17,80.76,exit rule
4,simple_day_trade,NVR,3/17/21,4589.17,2021-03-17,4662.33,exit rule
...,...,...,...,...,...,...,...
2305,simple_day_trade,UAA,11/26/21,23.73,2021-11-26,23.98,exit rule
2306,simple_day_trade,DXCM,11/29/21,584.04,2021-11-29,571.01,exit rule
2307,simple_day_trade,NET,11/29/21,193.39,2021-11-29,191.09,exit rule
2308,simple_day_trade,SGEN,11/29/21,167.33,2021-11-29,165.31,exit rule


In [409]:
#create a filter for dates greater than the start date and smaller than the end date
#we could change this based on the last data in the file
mask1 = (dfa['DateOut'] > '2021-11-26') & (dfa['DateOut'] <= '2021-11-29')
dfa_recent = dfa.loc[mask1]
dfa_recent = dfa_recent.sort_values(by=['Strategy','Symbol'], ascending=[False,True])

#same for model
mask2 = (dfm['DateOut'] > '2021-11-26') & (dfm['DateOut'] <= '2021-11-29')
dfm_recent = dfm.loc[mask2]
dfm_recent = dfm_recent.sort_values(by=['Strategy', 'Symbol'], ascending=[False,True])



In [410]:
#sort on model to find trades missed in real trades
df_sbs1 = pd.merge(dfm_recent, dfa_recent, how ='left',suffixes=["_M","_A"],  on=['Symbol','Strategy'])
df_sbs1 = df_sbs1.style.applymap(lambda x: '' if x==x else 'background-color: pink')
df_sbs1

,Strategy,Symbol,DateIn_M,PriceIn_M,DateOut_M,PriceOut_M,Reason_M,DateIn_A,PriceIn_A,DateOut_A,PriceOut_A,Reason_A
0,simple_day_trade,DXCM,11/29/21,584.040000,2021-11-29 00:00:00,571.010000,exit rule,nan,nan,NaT,nan,nan
1,simple_day_trade,NET,11/29/21,193.390000,2021-11-29 00:00:00,191.090000,exit rule,nan,nan,NaT,nan,nan
2,simple_day_trade,SGEN,11/29/21,167.330000,2021-11-29 00:00:00,165.310000,exit rule,nan,nan,NaT,nan,nan
3,simple_day_trade,U,11/29/21,172.150000,2021-11-29 00:00:00,170.330000,exit rule,nan,nan,NaT,nan,nan


In [411]:
#sort on actual trades to to find ones missing in the model
df_sbs2 = pd.merge(dfa_recent, dfm_recent,suffixes=["_A","_M"], how ='left',  on=['Symbol','Strategy'])
df_sbs2 = df_sbs2.style.applymap(lambda x: '' if x==x else 'background-color: pink')

df_sbs2

,Strategy,Symbol,DateIn_A,PriceIn_A,DateOut_A,PriceOut_A,Reason_A,DateIn_M,PriceIn_M,DateOut_M,PriceOut_M,Reason_M
0,simple_day_trade,ALPN,11/29/21,10.870000,2021-11-29 00:00:00,10.410000,exit rule,nan,nan,NaT,nan,nan
1,simple_day_trade,APYX,11/29/21,14.180000,2021-11-29 00:00:00,13.330000,exit rule,nan,nan,NaT,nan,nan
2,simple_day_trade,AUPH,11/29/21,18.560000,2021-11-29 00:00:00,18.340000,exit rule,nan,nan,NaT,nan,nan
3,simple_day_trade,CUE,11/29/21,12.160000,2021-11-29 00:00:00,12.010000,exit rule,nan,nan,NaT,nan,nan
4,simple_day_trade,IONQ,11/29/21,22.760000,2021-11-29 00:00:00,24.080000,exit rule,nan,nan,NaT,nan,nan
5,simple_day_trade,OSTK,11/29/21,89.780000,2021-11-29 00:00:00,90.060000,exit rule,nan,nan,NaT,nan,nan
6,simple_day_trade,RDIB,11/29/21,27.820000,2021-11-29 00:00:00,27.160000,exit rule,nan,nan,NaT,nan,nan
7,simple_day_trade,RNA,11/29/21,23.150000,2021-11-29 00:00:00,21.920000,exit rule,nan,nan,NaT,nan,nan


In [412]:
#we will create a list of all unmatched trades
#first convert styler object back to data frame object
missed_real_trades = df_sbs1.data

#check for rows with NaN
is_NaN1 = missed_real_trades.isnull()
row_has_NaN1 = is_NaN1.any(axis=1)
rows_with_NaN1 = missed_real_trades[row_has_NaN1]
rows_with_NaN1

###same with model table
missed_model_trades = df_sbs2.data

#check for rows with NaN
is_NaN2 = missed_model_trades.isnull()
row_has_NaN2 = is_NaN2.any(axis=1)
rows_with_NaN2 = missed_model_trades[row_has_NaN2]

#join dataframes together
all_missed_trades = rows_with_NaN1.append(rows_with_NaN2)

all_missed_trades


,Strategy,Symbol,DateIn_M,PriceIn_M,DateOut_M,PriceOut_M,Reason_M,DateIn_A,PriceIn_A,DateOut_A,PriceOut_A,Reason_A
0,simple_day_trade,DXCM,11/29/21,584.04,2021-11-29,571.01,exit rule,NaN,NaN,NaT,NaN,NaN
1,simple_day_trade,NET,11/29/21,193.39,2021-11-29,191.09,exit rule,NaN,NaN,NaT,NaN,NaN
2,simple_day_trade,SGEN,11/29/21,167.33,2021-11-29,165.31,exit rule,NaN,NaN,NaT,NaN,NaN
3,simple_day_trade,U,11/29/21,172.15,2021-11-29,170.33,exit rule,NaN,NaN,NaT,NaN,NaN
0,simple_day_trade,ALPN,NaN,NaN,NaT,NaN,NaN,11/29/21,10.87,2021-11-29,10.41,exit rule
1,simple_day_trade,APYX,NaN,NaN,NaT,NaN,NaN,11/29/21,14.18,2021-11-29,13.33,exit rule
2,simple_day_trade,AUPH,NaN,NaN,NaT,NaN,NaN,11/29/21,18.56,2021-11-29,18.34,exit rule
3,simple_day_trade,CUE,NaN,NaN,NaT,NaN,NaN,11/29/21,12.16,2021-11-29,12.01,exit rule
4,simple_day_trade,IONQ,NaN,NaN,NaT,NaN,NaN,11/29/21,22.76,2021-11-29,24.08,exit rule
5,simple_day_trade,OSTK,NaN,NaN,NaT,NaN,NaN,11/29/21,89.78,2021-11-29,90.06,exit rule
